In [1]:
import pickle 
import numpy as np 
import pandas as pd
import networkx as nx
from tqdm import tqdm
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import collections

from src.utils.data_utils import *
from src.utils.general_utils import *

C:\Users\pierr\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
movie_path = 'data/Movie.pkl'
actors_path = 'data/Actor.pkl'

with open(movie_path, 'rb') as file:
    Movie = pickle.load(file)

with open(actors_path, 'rb') as file:
    Actor = pickle.load(file)

C:\Users\pierr\AppData\Local\Temp\ipykernel_1608\4209001929.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  Movie = pickle.load(file)


In [25]:
Actor["actor_age_atmovierelease"] = Actor["actor_age_atmovierelease"].apply(
    lambda x: [pd.NA if pd.isna(age) else int(age) for age in x]
)

Actor["actor_age_atmovierelease"] = Actor["actor_age_atmovierelease"].apply(
    lambda x: [-1 if (pd.isna(val) or val < 0) else val for val in x]
)

Actor["Career_Start_age"] = Actor.apply(
    lambda row: min([val for val in row["actor_age_atmovierelease"] if val > 0], default=-1),
    axis=1,
)

max_career_years = int(max(Actor.apply(lambda row: max(row["actor_age_atmovierelease"]),axis=1)-Actor["Career_Start_age"]))

year_columns = [f"Year_{i}" for i in range(max_career_years)]
Actor_career = pd.concat(
    [Actor.copy(), pd.DataFrame(0, index=Actor.index, columns=year_columns)], axis=1)


In [26]:
def fill_career_years(row):

    age_at_release = np.array(row["actor_age_atmovierelease"])
    counts = np.zeros(max_career_years+1)
    
    if len(age_at_release) != 0:
        for year in age_at_release:
            if year >= 0 and row["Career_Start_age"]>=0:
                counts[year-row["Career_Start_age"]] += 1
                    
    return counts

In [51]:
start = time.time()
movie_releasedates = range(1905,2024)

print("Created graph with actors:")

for date in movie_releasedates+1:
    played_together = create_actor_network(Actor_career,Movie,1, min_releasedate =date) 

    print(f"Network ({date}/2024) of {played_together.number_of_nodes()} nodes.")

    #remove the nodes that are nans                
    nan_nodes = [node for node in played_together.nodes if pd.isna(node)] 
    played_together.remove_nodes_from(nan_nodes)
    
    #extract metrics from the network
    degrees = dict(played_together.degree())
    Actor_career[f"Degree_{date}"] = Actor_career["Freebase_actor_ID"].map(degrees)

    eigenvector_centrality = nx.eigenvector_centrality(played_together, max_iter=1000) 
    Actor_career[f"eigenvector_centrality_{date}"] = Actor_career["Freebase_actor_ID"].map(eigenvector_centrality)

    print(f"max degree: {Actor_career[f'Degree_{date}'].max()}")
    print(f"max centrality: {Actor_career[f'eigenvector_centrality_{date}'].max()}")

career_counts = np.array(Actor_career.apply(fill_career_years, axis=1).tolist())

for i in range(max_career_years):
    Actor_career[f"Year_{i+1}"] = career_counts[:, i]
    
print(f"Computation time: {time.time()-start:.2f}s")





Created graph with actors:


Creating network: 100%|██████████| 64308/64308 [00:01<00:00, 32536.47it/s]


Network (1906/2024) of 11 nodes.
max degree: 10.0
max centrality: 0.30151134457776363


Creating network: 100%|██████████| 64308/64308 [00:01<00:00, 37325.88it/s]


Network (1907/2024) of 15 nodes.
max degree: 10.0
max centrality: 0.301511344576197


...

Network (2023/2024) of 119365 nodes.


C:\Users\pierr\AppData\Local\Temp\ipykernel_1608\109354668.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career[f"eigenvector_centrality_{date}"] = Actor_career["Freebase_actor_ID"].map(eigenvector_centrality)


max degree: 1147.0
max centrality: 0.06482497866837857


C:\Users\pierr\AppData\Local\Temp\ipykernel_1608\109354668.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Actor_career[f"Year_{i+1}"] = career_counts[:, i]


NameError: name 'start' is not defined

In [57]:
date = 2024
Actor_career[Actor_career[f'Degree_{date}'] == Actor_career[f'Degree_{date}'].max()]

KeyError: 'Degree_2024'